In [3]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
!pip install pytorch-ood

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.1/120.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [5]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10, SVHN, Places365

from pytorch_ood.dataset.img import (
    LSUNCrop,
    Textures,
    TinyImageNetCrop,
    GaussianNoise
)
from pytorch_ood.detector import (
    ODIN,
    EnergyBased,
    Entropy,
    KLMatching,
    Mahalanobis,
    MaxLogit,
    MaxSoftmax,
    ViM,
)
from pytorch_ood.model import WideResNet
from pytorch_ood.utils import OODMetrics, ToRGB, ToUnknown

torch.manual_seed(1234)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([
    tvt.Resize(size=(32, 32)),
    ToRGB(),
    tvt.ToTensor(),
    tvt.Normalize(std=std, mean=mean)
])

In [ ]:
def calculate(test_loader, detector):
    metrics = OODMetrics() # Evaluate Detectors

    print("Total count:", len(test_loader))
    cnt = 0
    for x, y in test_loader:
        metrics.update(detector(x.cuda()), y)
        cnt += 1
        if (cnt + 1) % 50 == 0:
            print(f"[{cnt}/{len(test_loader)}]")

    r = metrics.compute()
    print(r)
    return r

In [5]:
def test_with_all_datasets(detector, base_dataset, transform=trans, batch_size=128, num_workers=20):
    dataset_names = ["SVHN", "Textures", "LSUNCrop", "TinyImageNetCrop", "Places365", "GaussianNoise"]
    results = []

    for name in dataset_names:
        if name == "SVHN":
            dataset_out_test = SVHN(root="data", split="test", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "Textures":
            dataset_out_test = Textures(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "LSUNCrop":
            dataset_out_test = LSUNCrop(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "TinyImageNetCrop":
            dataset_out_test = TinyImageNetCrop(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "Places365":
            dataset_out_test = Places365(root="data", split="val", small=True, transform=transform, target_transform=ToUnknown(), download=True)
        elif name == "GaussianNoise":
            dataset_out_test = GaussianNoise(length=10000, transform=transform, target_transform=ToUnknown())
        else:
            continue

        test_loader = DataLoader(torch.utils.data.ConcatDataset([base_dataset, dataset_out_test]), batch_size=batch_size, num_workers=num_workers)
        print()
        print(name)
        results.append(calculate(test_loader, detector))

    return results

### MaxSoftmax

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = MaxSoftmax(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

### ODIN

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = ODIN(model, eps=0.002)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9162966012954712, 'AUPR-IN': 0.959459125995636, 'AUPR-OUT': 0.8529837131500244, 'FPR95TPR': 0.3149000108242035}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.883427083492279, 'AUPR-IN': 0.8069800138473511, 'AUPR-OUT': 0.924010694026947, 'FPR95TPR': 0.482699990272522}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9748982787132263, 'AUPR-IN': 0.9730662107467651, 'AUPR-OUT': 0.978147566318512, 'FPR95TPR': 0.11389999836683273}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9580070376396179, 'AUPR-IN': 0.9518746137619019, 'AUPR-OUT': 0.9649056792259216, 'FPR95TPR': 0.15440000593662262}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.8792003989219666, 'AUPR-IN': 0.9568765759468079, 'AUPR-OUT': 0.6914

### Mahalanobis

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = Mahalanobis(model.features)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9681909680366516, 'AUPR-IN': 0.9873345494270325, 'AUPR-OUT': 0.9281249046325684, 'FPR95TPR': 0.17970000207424164}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.9626320600509644, 'AUPR-IN': 0.9454406499862671, 'AUPR-OUT': 0.9771375060081482, 'FPR95TPR': 0.20759999752044678}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9368303418159485, 'AUPR-IN': 0.9226568341255188, 'AUPR-OUT': 0.9476874470710754, 'FPR95TPR': 0.23520000278949738}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.868048906326294, 'AUPR-IN': 0.8204649090766907, 'AUPR-OUT': 0.8959041237831116, 'FPR95TPR': 0.3774999976158142}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.8016583919525146, 'AUPR-IN': 0.9231971502304077, 'AUPR-OUT': 0

### Energy

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = EnergyBased(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9107781052589417, 'AUPR-IN': 0.9603055119514465, 'AUPR-OUT': 0.7920160889625549, 'FPR95TPR': 0.43869999051094055}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.8534713387489319, 'AUPR-IN': 0.7847411632537842, 'AUPR-OUT': 0.8866969347000122, 'FPR95TPR': 0.6661999821662903}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9905964136123657, 'AUPR-IN': 0.9902125597000122, 'AUPR-OUT': 0.9913669228553772, 'FPR95TPR': 0.04520000144839287}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9797328114509583, 'AUPR-IN': 0.9775981903076172, 'AUPR-OUT': 0.9819805026054382, 'FPR95TPR': 0.09070000052452087}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.8988311290740967, 'AUPR-IN': 0.9675583243370056, 'AUPR-OUT': 

### Entropy

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = Entropy(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9239158630371094, 'AUPR-IN': 0.9631484746932983, 'AUPR-OUT': 0.8661899566650391, 'FPR95TPR': 0.28760001063346863}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.8883779048919678, 'AUPR-IN': 0.8038489818572998, 'AUPR-OUT': 0.9307792782783508, 'FPR95TPR': 0.4153999984264374}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9732518196105957, 'AUPR-IN': 0.9713162779808044, 'AUPR-OUT': 0.9768692851066589, 'FPR95TPR': 0.11940000206232071}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9543061256408691, 'AUPR-IN': 0.9478659629821777, 'AUPR-OUT': 0.962076723575592, 'FPR95TPR': 0.16670000553131104}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.8870015740394592, 'AUPR-IN': 0.9603999257087708, 'AUPR-OUT': 0

### MaxLogit

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = MaxLogit(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9110583662986755, 'AUPR-IN': 0.9607238173484802, 'AUPR-OUT': 0.7922630906105042, 'FPR95TPR': 0.43849998712539673}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.8548011183738708, 'AUPR-IN': 0.7865685224533081, 'AUPR-OUT': 0.8875477910041809, 'FPR95TPR': 0.6614999771118164}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9895871877670288, 'AUPR-IN': 0.9889260530471802, 'AUPR-OUT': 0.9906085729598999, 'FPR95TPR': 0.04859999939799309}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.978347897529602, 'AUPR-IN': 0.9755653142929077, 'AUPR-OUT': 0.9810075163841248, 'FPR95TPR': 0.0949999988079071}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.8981443047523499, 'AUPR-IN': 0.9671193957328796, 'AUPR-OUT': 0.

### KLMatching

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = KLMatching(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.8434356451034546, 'AUPR-IN': 0.9250538945198059, 'AUPR-OUT': 0.6879284381866455, 'FPR95TPR': 0.4316999912261963}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.7598135471343994, 'AUPR-IN': 0.698091983795166, 'AUPR-OUT': 0.769401490688324, 'FPR95TPR': 0.9681000113487244}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.90835040807724, 'AUPR-IN': 0.9097596406936646, 'AUPR-OUT': 0.8423641324043274, 'FPR95TPR': 0.43380001187324524}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8362108469009399, 'AUPR-IN': 0.8642855882644653, 'AUPR-OUT': 0.7221440076828003, 'FPR95TPR': 0.9800000190734863}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.7207571268081665, 'AUPR-IN': 0.915798008441925, 'AUPR-OUT': 0.30891

### ViM

In [ ]:
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = ViM(model.features, d=64, w=model.fc.weight, b=model.fc.bias)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.955865204334259, 'AUPR-IN': 0.9816233515739441, 'AUPR-OUT': 0.9008862376213074, 'FPR95TPR': 0.23499999940395355}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.9378482699394226, 'AUPR-IN': 0.9089112281799316, 'AUPR-OUT': 0.9596679210662842, 'FPR95TPR': 0.3102000057697296}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9848831295967102, 'AUPR-IN': 0.9845131039619446, 'AUPR-OUT': 0.9858312010765076, 'FPR95TPR': 0.07689999788999557}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.958219051361084, 'AUPR-IN': 0.9539929628372192, 'AUPR-OUT': 0.9625090956687927, 'FPR95TPR': 0.18799999356269836}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.8559735417366028, 'AUPR-IN': 0.9481277465820312, 'AUPR-OUT': 0.6512577533721924, 'FPR95TPR': 0.5353999733924866}

GaussianNoise
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC

### Outlier Exposure (OE)

In [29]:
from torch.optim import Adam
from pytorch_ood.dataset.img import Textures, TinyImages300k
from pytorch_ood.detector import MaxSoftmax
from pytorch_ood.loss import OutlierExposureLoss
from pytorch_ood.utils import OODMetrics, ToUnknown

torch.manual_seed(123)

# maximum number of epochs and training iterations
n_epochs = 10
device = "cuda:0"

# %%
# Setup preprocessing and data
trans = tvt.Compose([tvt.Resize(size=(32, 32)), tvt.ToTensor()])

# setup IN training data
dataset_in_train = CIFAR10(root="data", train=True, download=True, transform=trans)

# setup OOD training data, use ToUnknown() to mark labels as OOD
# this way, outlier exposure can automatically decide if the training samples are IN or OOD
dataset_out_train = TinyImages300k(
    root="data", download=True, transform=trans, target_transform=ToUnknown()
)

# setup IN test data
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)

# setup OOD test data, use ToUnknown() to mark labels as OOD
dataset_out_test = Textures(
    root="data", download=True, transform=trans, target_transform=ToUnknown()
)

# create data loaders
train_loader = DataLoader(
    dataset_in_train + dataset_out_train, batch_size=64, shuffle=True
)
test_loader = DataLoader(dataset_in_test + dataset_out_test, batch_size=64)

# %%
# Create DNN, pretrained on the imagenet excluding cifar10 classes
model = WideResNet(num_classes=1000, pretrained="imagenet32-nocifar")
# we have to replace the final layer to account for the lower number of
# classes in the CIFAR10 dataset
model.fc = torch.nn.Linear(model.fc.in_features, 10)

model.to(device)

opti = Adam(model.parameters())
criterion = OutlierExposureLoss(alpha=0.5)

# %%
# Define a function to test the model
def test(test_loader):
    softmax = MaxSoftmax(model)

    metrics_softmax = OODMetrics()
    model.eval()

    with torch.no_grad():
        for x, y in test_loader:
            metrics_softmax.update(softmax(x.to(device)), y)

    print(metrics_softmax.compute())
    model.train()

# %%
# Start training
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    for x, y in train_loader:
        logits = model(x.to(device))
        loss = criterion(logits, y.to(device))
        opti.zero_grad()
        loss.backward()
        opti.step()

    test(test_loader)

Files already downloaded and verified
Epoch 0
{'AUROC': 0.9495147466659546, 'AUPR-IN': 0.9036620259284973, 'AUPR-OUT': 0.9695314168930054, 'FPR95TPR': 0.21230000257492065}
Epoch 1
{'AUROC': 0.9774715900421143, 'AUPR-IN': 0.9473906755447388, 'AUPR-OUT': 0.9878228902816772, 'FPR95TPR': 0.08959999680519104}
Epoch 2
{'AUROC': 0.9715052843093872, 'AUPR-IN': 0.9269547462463379, 'AUPR-OUT': 0.9862478375434875, 'FPR95TPR': 0.09030000120401382}
Epoch 3
{'AUROC': 0.9716963768005371, 'AUPR-IN': 0.9460590481758118, 'AUPR-OUT': 0.9844248294830322, 'FPR95TPR': 0.12030000239610672}
Epoch 4
{'AUROC': 0.9803022742271423, 'AUPR-IN': 0.9636546969413757, 'AUPR-OUT': 0.9886083602905273, 'FPR95TPR': 0.09269999712705612}
Epoch 5
{'AUROC': 0.9829574227333069, 'AUPR-IN': 0.9608777761459351, 'AUPR-OUT': 0.9902729988098145, 'FPR95TPR': 0.06319999694824219}
Epoch 6
{'AUROC': 0.9811888933181763, 'AUPR-IN': 0.9580562710762024, 'AUPR-OUT': 0.989603579044342, 'FPR95TPR': 0.07519999891519547}
Epoch 7
{'AUROC': 0.98817

In [30]:
def test(test_loader, name):
    softmax = MaxSoftmax(model)

    metrics_softmax = OODMetrics()
    model.eval()

    with torch.no_grad():
        for x, y in test_loader:
            metrics_softmax.update(softmax(x.to(device)), y)
    print(name)
    print(metrics_softmax.compute())

In [31]:
dataset_names = ["SVHN", "Textures", "LSUNCrop", "TinyImageNetCrop", "Places365", "GaussianNoise"]
datasets_out_test = {
    "SVHN": SVHN(root="./data", split="test", download=True, transform=trans, target_transform=ToUnknown()),
    "Textures": Textures(root="./data", download=True, transform=trans, target_transform=ToUnknown()),
    "LSUNCrop": LSUNCrop(root="./data", download=True, transform=trans, target_transform=ToUnknown()),
    "TinyImageNetCrop": TinyImageNetCrop(root="./data", download=True, transform=trans, target_transform=ToUnknown()),
    "Places365": Places365(root="./data", split="val", small=True, download=False, transform=trans, target_transform=ToUnknown()),
    "GaussianNoise": GaussianNoise(length=10000, transform=trans, target_transform=ToUnknown())
}

for name, dataset in datasets_out_test.items():
    dataset_in_test =  CIFAR10(root="data", train=False, transform=trans)
    test_loader = DataLoader(dataset_in_test + dataset, batch_size=64)
    test(test_loader, name)

Using downloaded and verified file: ./data/test_32x32.mat
SVHN
{'AUROC': 0.9942606091499329, 'AUPR-IN': 0.9970419406890869, 'AUPR-OUT': 0.9880416989326477, 'FPR95TPR': 0.020500000566244125}
Textures
{'AUROC': 0.9852719306945801, 'AUPR-IN': 0.9724277257919312, 'AUPR-OUT': 0.9915739297866821, 'FPR95TPR': 0.06520000100135803}
LSUNCrop
{'AUROC': 0.9927982091903687, 'AUPR-IN': 0.9852676391601562, 'AUPR-OUT': 0.9944685101509094, 'FPR95TPR': 0.01889999955892563}
TinyImageNetCrop
{'AUROC': 0.9780853986740112, 'AUPR-IN': 0.9730455279350281, 'AUPR-OUT': 0.9788156747817993, 'FPR95TPR': 0.10689999908208847}
Places365
{'AUROC': 0.9685079455375671, 'AUPR-IN': 0.9905447363853455, 'AUPR-OUT': 0.8906952142715454, 'FPR95TPR': 0.15760000050067902}
GaussianNoise
{'AUROC': 0.9975572824478149, 'AUPR-IN': 0.9930383563041687, 'AUPR-OUT': 0.9985091090202332, 'FPR95TPR': 0.004900000058114529}
